In [2]:
# Define some colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
BLUE = (0, 0, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)

WIDTH = 22
HEIGHT = 22
MARGIN = 3

In [ ]:
import math
import pygame

class Field:
    def __init__(self, x, y, color=WHITE):
        self.x = x
        self.y = y
        self.color = color
        self.g = float('inf')
        self.h = 0
        self.f = float('inf')
        self.parent = None

    def draw(self, screen):
        pygame.draw.rect(screen,
                         self.color,
                         [(MARGIN + WIDTH) * self.y + MARGIN,
                          (MARGIN + HEIGHT) * self.x + MARGIN,
                          WIDTH,
                          HEIGHT])
    
    def __lt__(self, other):
        return self.f < other.f

class Grid:
    def __init__(self, rows, cols):
        self.rows = rows
        self.cols = cols
        self.grid = [[Field(x, y) for y in range(cols)] for x in range(rows)]
        self.start = None
        self.goal = None

    def draw(self, screen):
        for row in self.grid:
            for field in row:
                draw_x = field.y
                draw_y = self.rows - 1 - field.x  # flip vertically
                pygame.draw.rect(screen,
                                 field.color,
                                 [(MARGIN + WIDTH) * draw_x + MARGIN,
                                  (MARGIN + HEIGHT) * draw_y + MARGIN,
                                  WIDTH,
                                  HEIGHT])

    def set_obstacles(self, positions):
        for y, x in positions:
            self.grid[y][x].color = BLACK
    
    def calcH(self, field):
        return math.sqrt((field.x - self.goal.x) ** 2 + (field.y - self.goal.y) ** 2)
    
    def get_neighbors(self, field):
        neighbors = []
        directions = [(-1,0), (1,0), (0,-1), (0,1)]  # up, down, left, right
        for dx, dy in directions:
            x = field.x + dx
            y = field.y + dy
            if 0 <= x <= self.rows-1 and 0 <= y <= self.cols-1: #staying in the grid
                neighbor = self.grid[x][y]
                if neighbor.color != BLACK:
                    neighbors.append(neighbor)
        return neighbors

In [8]:
class Queue:
    def __init__(self, mode='FIFO'):
        assert mode in ['FIFO', 'LIFO', 'PRIO'], "Mode must be 'FIFO', 'LIFO', or 'PRIO'"
        self.mode = mode
        self.items = []

    def push(self, item):
        if self.mode in ['FIFO', 'LIFO']:
            self.items.append(item)
        elif self.mode in ['PRIO']:
            self.items.append(item)
            self.items.sort(key=lambda x: x.f)
        
    def pop(self):
        if self.mode in ['FIFO', 'PRIO']:
            return self.items.pop(0)
        if self.mode in ['LIFO']:
            return self.items.pop()
                
    def is_empty(self):
        return len(self.items) == 0
    
    def contains(self, field):
         return any(item == field for item in self.items)
     
    def remove(self, field):
        for i, item in enumerate(self.items):
            if item == field:
                del self.items[i]
                break

In [ ]:
def initialize(grid, open):
    grid.start.g = 0
    grid.start.parent = grid.start
    grid.start.h = grid.calcH(grid.start)
    grid.start.f = grid.start.g + grid.start.h
    open.push(grid.start)
    
def upadateVertex(grid, current, neighbor, open):
    if current.g + 1 < neighbor.g:
        neighbor.g = current.g + 1
        neighbor.parent = current
        
        if open.contains(neighbor):
            open.remove(neighbor)
        
        neighbor.h = grid.calcH(neighbor)
        neighbor.f = neighbor.g + neighbor.h
        
        #Refresh the value
        open.push(neighbor)
                
def a_star(grid, open, closed):
    
    if not open.is_empty():
        current = open.pop()
        
        #Start and goal stay green and red
        if not current == grid.start and not current == grid.goal:
            current.color = BLUE
        if current == grid.goal:
            # Trace back the path
            path = []
            current_node = current.parent
            while current_node != grid.start:
                current_node.color = GREEN
                path.append(current_node)
                current_node = current_node.parent
            path.reverse()
            return path
        
        #Get all neighbors
        closed.push(current)
        current_neighbors = grid.get_neighbors(current)
        
        #For all neighbors
        for neighbor in current_neighbors:
            if not closed.contains(neighbor):
                upadateVertex(grid, current, neighbor, open)
            
    
    elif open.is_empty():
        print('NO PATH FOUND')
        return -1

In [ ]:
pygame.init()
size = (500, 500)
screen = pygame.display.set_mode(size)
pygame.display.set_caption("My Game")
done = False
path = None
path_is_printed = False
clock = pygame.time.Clock()
grid = Grid(20, 20)

walls = [
    *[(y, 9) for y in range(0, 10)],      # vertical wall at column 10
    *[(9, x) for x in range(4, 10)],      # horizontal wall at row 10
    *[(y, 16) for y in range(9, 20)]      # vertical wall at column 18
]
grid.set_obstacles(walls)

# Start and goal
start = grid.grid[0][0]
goal = grid.grid[19][19]
grid.grid[0][0].color = GREEN     # S at (1,1)
grid.grid[19][19].color = RED     # G at (20,20)
grid.start = grid.grid[0][0]
grid.goal = grid.grid[19][19]

open = Queue('PRIO')
closed = Queue()
    
initialize(grid, open)

while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            done = True
        
        if path is None:    
            path = a_star(grid, open, closed)       
            
        screen.fill(BLACK)
        grid.draw(screen)

        pygame.display.flip()
        clock.tick(60)
        
        if path is not None and not path_is_printed:
            print("Path:")
            for node in path:
                print(f"({node.x}, {node.y})")
            done = True    
            path_is_printed = True
pygame.quit()

Path:
(1, 0)
(1, 1)
(2, 1)
(2, 2)
(3, 2)
(3, 3)
(4, 3)
(5, 3)
(6, 3)
(7, 3)
(8, 3)
(9, 3)
(10, 3)
(10, 4)
(10, 5)
(10, 6)
(10, 7)
(10, 8)
(10, 9)
(10, 10)
(10, 11)
(10, 12)
(10, 13)
(10, 14)
(10, 15)
(9, 15)
(8, 15)
(8, 16)
(8, 17)
(9, 17)
(10, 17)
(11, 17)
(12, 17)
(13, 17)
(14, 17)
(15, 17)
(16, 17)
(17, 17)
(18, 17)
(18, 18)
(19, 18)


: 